In [1]:
!git clone https://github.com/derInformatiker/AIcrowd-AIBlitz7-Solution.git
!pip install -r AIcrowd-AIBlitz7-Solution/challenge4/requirements.txt
!pip install aicrowd-cli==0.1

Cloning into 'AIcrowd-AIBlitz7-Solution'...
remote: warning: multi-pack bitmap is missing required reverse index
remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 317 (delta 186), reused 219 (delta 121), pack-reused 0
Receiving objects: 100% (317/317), 469.43 KiB | 18.78 MiB/s, done.
Resolving deltas: 100% (186/186), done.
     |████████████████████████████████| 10.1MB 17.8MB/s 
     |████████████████████████████████| 28.2MB 112kB/s 
     |████████████████████████████████| 829kB 45.9MB/s 
     |████████████████████████████████| 122kB 51.3MB/s 
     |████████████████████████████████| 829kB 46.9MB/s 
     |████████████████████████████████| 112kB 53.3MB/s 
     |████████████████████████████████| 276kB 61.4MB/s 
     |████████████████████████████████| 952kB 41.3MB/s 
     |████████████████████████████████| 1.3MB 31.7MB/s 
     |████████████████████████████████| 143kB 47.8MB/s 
     |█████

     |████████████████████████████████| 40kB 2.3MB/s 
     |████████████████████████████████| 163kB 17.9MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 204kB 38.8MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for aicrowd-cli: filename=aicrowd_cli-0.1.0-cp37-none-any.whl size=34929 sha256=4b79e7fd91a34e7d84e022bb6056132d92548bb51cdbfd6b0552678eec3ba179
  Stored in directory: /root/.cache/pip/wheels/7e/ec/cc/a38391f23ad9bd044c91cd5e6e0a30f2a4265a1eb5adedf9f5
Successfully built aicrowd-cli


###RESTART RUNTIME TO USE NEW PACKAGES

In [1]:
API_KEY = ""  # Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [14]:
!aicrowd dataset download --challenge stage-prediction

!rm -rf data
!mkdir data

!unzip -q test.zip  -d data
import os
os.rename('data/test', 'data/train')
!mv sample_submission.csv data/sample_submission.csv

sample_submission.csv: 100% 13.3k/13.3k [00:00<00:00, 796kB/s]
test.zip: 100% 27.7M/27.7M [00:02<00:00, 10.6MB/s]


In [15]:
import shutil

shutil.copy('AIcrowd-AIBlitz7-Solution/challenge4/model.py','model.py')
shutil.copy('AIcrowd-AIBlitz7-Solution/challenge4/dataset.py','dataset.py')

'dataset.py'

In [16]:
import pandas as pd
baseline = []
images_list = [598,1048,1198,1498,1798]
last = 0
for u, i in enumerate(images_list):
    baseline.extend([u]*(i-last))
    last = i
baseline = [0]+baseline

df = pd.read_csv('data/sample_submission.csv')
df.label = baseline
df.to_csv('data/labels.csv',index = False)

In [17]:
import model
import dataset
import torch
import albumentations as A
import pytorch_lightning as pl
from pytorch_lightning import Trainer

if __name__ == '__main__':
    trainer = Trainer(max_epochs = 30,gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True)
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)
    output = torch.tensor([])

    for i in range(len(out)):
        output = torch.cat((output,torch.tensor(out[i][0]).argmax(1)))
    output = output.int().tolist()
    dataset.writeSub(output)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 10.7 M
----------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.816    Total estimated model params size (MB)


Loaded pretrained weights for efficientnet-b3


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


/content/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: you defined a test_step but have no test_dataloader. Skipping test loop
  warnings.warn(*args, **kwargs)
